## 03a - Assess Unique ILLs 
go through and get the number of outlinks between version

import:

    00_en_as_rev_everything.pkl
    00_ar_as_rev_everything.pkl
    
output:

    03a_onlyEnNoILL.txt
    sizeOverTime_AS_ar.png
    sizeOverTime_AS_BOTH.pdf
    
    
The Plan:
- this goes through and does some initial work with the outlinks 
- it grabs all of the outlinks in the english and arabic versions of the arab spring
- organize them by ones that are only in each language. 
- identify ones that are in both by if there were english outlinks
- that had an arabic ILL and if that arabic ILL was also an outlink


1. Get Outlinks
2. Check ILL Symmetry - English
3. Check ILL Symmetry - Arabic
4. Table



Ouput: 

there are 327 unique english outlinks
there are 269 arabic outlinks

There are 79 outlinks shared between the two articles

There are 18 links that are inly in English with no Arabic ILLs
There are 21 links that are only in Arabic with no English ILLs

There are 230 english outlinks that have Arabic ILLs but those ILLs are not links referenced in the arabic article
There are 169 arabic outlinks that have English ILLs but those ILLs are not links referenced in the english articl


In [2]:

import numpy as np
import pandas as pd

import wikifunctions as wf #Brian's stuff

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

import json, requests

from urllib.parse import unquote

## 1. Get Outlinks

In [3]:
#English
out_en_lyst = wf.get_page_outlinks('Arab Spring')
print(len(out_en_lyst))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#Arabic
out_ar_lyst = wf.get_page_outlinks('الربيع العربي', endpoint='ar.wikipedia.org/w/api.php')
print(len(out_ar_lyst))

499


### 1a. Resolve Redirects

In [ ]:
out_en_lyst_noReDir = wf.resolve_redirects(out_en_lyst)
out_ar_lyst_noReDir = wf.resolve_redirects(out_ar_lyst,endpoint='ar.wikipedia.org/w/api.php')

print(len(out_en_lyst_noReDir))
print(len(out_ar_lyst_noReDir ))

### 1b. Delete Duplicates

In [ ]:
#get rid of duplicates in English
out_en_lyst_clean = []

for x in out_en_lyst_noReDir:
    if x not in out_en_lyst_clean: 
        out_en_lyst_clean.append(x)
        
#    else:
#         print(x)

print(len(out_en_lyst_clean))

323


In [ ]:
#get rid of duplicates Arabic
out_ar_lyst_clean = []

# [out_en_lyst_clean_noDups.append(x) for x in out_en_lyst_clean if x not in out_en_lyst_clean_noDups]

for x in out_ar_lyst_noReDir:
    if x not in out_ar_lyst_clean: 
        out_ar_lyst_clean.append(x)
        
#     else:
#         print(x)

print(len(out_ar_lyst_clean))

271


## Get ILLs of a page

## 2. Check ILL Symmetry - English

go through ILLs of outlinks and see if the ILL is in the outlink of the other language (english)

In [ ]:
onlyEnNoILL = [] #a list of english links - this is an En ILL but no arabic ILL 
enOutarILL = [] #a list of english links - there is an arILL in out_ar_lyst_clean
onlyEn = [] #a list of english links- that have an arabic out link but it is not in in out_ar_lyst_clean


go through english outlinks get the ones that have arabic ILLs and where there are none

In [ ]:
#gets all of the arabic ILL for english outlinks
#if there is one it is added to:
arILLforEnOutlinks = []

#if there isn't one it is added to: 
onlyEnNoILL = [] 

for enOut in out_en_lyst_clean:
    ILL = wf.get_interlanguage_links(enOut)
    try:
        arILL = ILL['ar']
        arILLforEnOutlinks.append(arILL)
        #print(arILL)
    except KeyError as ke:
        #print(enOut)
        onlyEnNoILL.append(enOut) #no arabic ILL 
    

In [ ]:
#checks to see if the arabic ILLs are in the outlinks of the arabic page

#if they are add it to: 
enOutarILL = []
enOutarILL_arabic = []

#if the arabic ILL is not referenced also in the arabic page add it to
onlyEn = []
onlyEn_english = []

        
for arOut in arILLforEnOutlinks: #goes through all of the arabic outlinks
    ILL = wf.get_interlanguage_links(arOut, 'ar.wikipedia.org/w/api.php' )
    #print(ILL)
    enILL = ILL['en'] #goes backward to the english ILL
        
    if arOut in out_ar_lyst_clean: #there is arILL in the arabic outlinks 
        enOutarILL.append(enILL)
        enOutarILL_arabic.append(arOut)
        
    else:
        onlyEn.append(arOut) #this is if it has an arILl but it isn't in the outlinks
        onlyEn_english.append(enILL)
        

In [ ]:
print(len(onlyEnNoILL))
print(len(enOutarILL))
print(len(enOutarILL_arabic))
print(len(onlyEn)) 

15
79
79
229


In [ ]:
len(out_en_lyst_clean)

323

Save Data

In [ ]:
with open("03a_onlyEnNoILL.txt", "w") as f:
    for s in onlyEnNoILL:
        f.write(str(s) +"\n")

In [ ]:
with open("03a_enOutarILL.txt", "w") as f:
    for s in enOutarILL:
        f.write(str(s) +"\n")

In [ ]:
with open("03a_onlyEn.txt", "w") as f:
    for s in onlyEn:
        f.write(str(s) +"\n")

In [ ]:
with open("03a_onlyEn_english.txt", "w") as f:
    for s in onlyEn_english:
        f.write(str(s) +"\n")

## 3. Check ILL Symmetry - Arabic

go through arabic outlinks

go through Arabic outlinks get the ones that have English ILLs and where there are none

In [ ]:
#gets all of the english translations of the outlinks
#if there is an english ILL
enILLforArOutlinks = []
#if there isn't one it is added to: 
onlyArNoILL = [] 

for arOut in out_ar_lyst_clean: #goes throguh all the arabic outlines
    ILL = wf.get_interlanguage_links(arOut, endpoint='ar.wikipedia.org/w/api.php')
    try:
        enILL = ILL['en']
        enILLforArOutlinks.append(enILL) #gets english ILL
        #print(arILL)
    except KeyError as ke:
        onlyArNoILL.append(arOut) #no english ILL 
        

        

In [ ]:
#checks to see if the engish ILLs are in the outlinks of the english page

#if they are add it to: 
arOutenILL = []
arOutenILL_english = []

#if the english ILL is not referenced also in the arabic page add it to
onlyAr = []
onlyAr_arabic = []
        
for enOut in enILLforArOutlinks: #goes through all the ones with ILLs in english
    ILL = wf.get_interlanguage_links(enOut, endpoint='ar.wikipedia.org/w/api.php')
    arILL = ILL['ar'] #translates them back to arabic
    #print(arILL)
        
    if enOut in out_en_lyst_clean: #there is enILL in the english outlinks 
        arOutenILL.append(arILL) #adds the arabic version
        arOutenILL_english.append(enOut) #keept the english version
        
    else:
        onlyAr.append(enOut) #this is if it has an arILl but it isn't in the outlinks
        onlyAr_arabic.append(arILL)

In [ ]:
print(len(onlyArNoILL))
print(len(arOutenILL))
print(len(arOutenILL_english))
print(len(onlyAr)) 

21
79
79
171


In [ ]:
print(len(out_ar_lyst_clean))

269


Save Data

In [ ]:
with open("03a_onlyAr.txt", "w") as f:
    for s in onlyAr:
        f.write(str(s) +"\n")

In [ ]:
with open("03a_onlyAr_arabic.txt", "w") as f:
    for s in onlyAr_arabic:
        f.write(str(s) +"\n")

In [ ]:
with open("03a_onlyArNoILL.txt", "w") as f:
    for s in onlyArNoILL:
        f.write(str(s) +"\n")

In [ ]:
onlyAr

['January 14',
 'January 25',
 'October 20',
 'February 11',
 'Jeddah',
 'March 15',
 'February 17',
 '2010',
 '2011',
 'Corruption',
 'Palestine (region)',
 '2011 Lebanese protests',
 '2011 Israeli border demonstrations',
 'Battle of Sirte',
 'January 4',
 'Hafiz al-Assad',
 'Police',
 'Constitution',
 'December 17',
 'December 18',
 'Gamal Mubarak',
 'Liberty',
 '1969 Libyan revolution',
 'Bernard Lewis',
 '2012',
 '2014',
 'Constitutional monarchy',
 'Zionist entity',
 'Atlantic Ocean',
 'Ahmed Saleh',
 'Political demonstration',
 'January 15',
 'Alexandria',
 'Bullet',
 'Suez Governorate',
 'Reuters',
 "Coup d'état",
 'Martial law',
 'Fouad Mebazaa',
 'Chamber of Deputies (Tunisia)',
 'National Police Day (Egypt)',
 'Abdel Fattah el-Sisi',
 'June 2013 Egyptian protests',
 'Latakia',
 'Rif Dimashq Governorate',
 'March 18',
 'Bayda, Libya',
 'International Court of Justice',
 'International Criminal Court',
 'Muhammad Gaddafi',
 'Hala Misrati',
 'Atef Najib',
 '2003',
 'West Bank',


## make a table

In [ ]:
from tabulate import tabulate

In [ ]:
xlist = ['English', 'Arabic']
ylist = ['links with no ILL in other lang', 'link ILL but not in other article', 'shared ILLs']


In [ ]:
#print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

NameError: name 'table' is not defined

## go through and see if there are any ar outlinks that don't have an ILL to english 

In [ ]:
onlyArNoILL = []

for arOut in out_ar_lyst: #go through all the english words
    ILL = wf.get_interlanguage_links(arOut)
    try:
        enILL = ILL['en']
    except KeyError as ke:
        onlyArNoILL.append(arOut)

In [ ]:
print(onlyArNoILL)

In [ ]:
#print(onlyAr_clean)

## add the ones that don't have an ILL to the only EN page

In [ ]:
onlyEn_clean = onlyEn_clean + onlyEnNoILL

In [ ]:
print(len(onlyEn_clean))

## export to txt

In [ ]:
#inBoth_clean = []
#onlyEn_clean = []
#onlyAr_clean = []
#onlyEnNoILL

In [ ]:
with open(r'outlinks_AS_onlyEn.txt', 'w') as fp:
    for item in onlyEn_clean:
        # write each item on a new line
        fp.write("%s\n" % item)
    print('Done')